In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **이전 대화내용 기억하는 Agent**

`RunnableWithMessageHistory` 를 사용하여 `AgentExecutor` 를 감싸줍니다.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_teddynote.messages import AgentStreamParser

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader('data/SPRI_AI_Brief_2023년12월호_F.pdf')
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,              # 리트리버
    name='pdf_search',      # 도구의 이름
    
    # 도구에 대한 설명을 자세히 기입해야 합니다. 
    # LLM에게 알려주는 내용인데.. 이 툴을 언제 쓸지 언제 활용하면 좋을지를 영어로 작성합니다.
    description="use this tool to search information from the PDF document", 
)

In [8]:
search  = TavilySearchResults(k=6)      # 검색 결과를 6개까지 가져오겠다

In [9]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

In [10]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [11]:
prompt = ChatPromptTemplate.from_messages(                  # 프롬프트
    [
        (
            'system',
            'You are a helpful assistant. '
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ('placeholder', '{chat_history}'),
        ('human', '{input}'),
        ('placeholder', '{agent_scratchpad}'),
    ]
)

In [12]:
agent = create_tool_calling_agent(llm, tools, prompt)       # tool calling agent 생성

In [13]:
agent_executor = AgentExecutor(     
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

In [14]:
store = {}   

def get_session_history(session_ids):   # session_id 를 기반으로 세션 기록을 가져오는 함수    
    if session_ids not in store:        # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 딕셔너리에 저장
    
    return store[session_ids]           # 해당 세션 ID에 대한 세션 기록 리턴


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,                                    # session_id 를 기반으로 세션 기록을 가져오는 함수
    input_messages_key='input',                             # 프롬프트의 질문으로 들어가는 key: input
    history_messages_key='chat_history'                     # 프롬프트의 대화 기록이 들어가는 key: chat_history
)

In [15]:
response = agent_with_chat_history.stream(
    {'input': '삼성전자가 자체 개발한 생성형 AI에 대한 정보를 문서에서 검색해서 알려주세요.'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [16]:
for step in response:
    print('step:', step)
    print()
    
    if 'actions' in step:
        print('==' * 50)
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

        print('==' * 50)

    elif 'steps' in step:
        print('==' * 50)
        print(f'[steps]')

        for agent_step in step['steps']:
            print("Action:")
            print(f"  Tool: {agent_step.action.tool}")
            print(f"  Tool Input: {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")
        
        print('==' * 50)

    elif 'output' in step:
        print('==' * 50)
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")
                
        print('==' * 50)



> Entering new None chain...
step: {'actions': [ToolAgentAction(tool='pdf_search', tool_input={'query': '삼성전자 생성형 AI'}, log="\nInvoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_J890dCbL8vZdPIzZpyn697JL', 'function': {'arguments': '{"query":"삼성전자 생성형 AI"}', 'name': 'pdf_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1'}, id='run-1b418405-c09f-497c-91a6-e66339845662', tool_calls=[{'name': 'pdf_search', 'args': {'query': '삼성전자 생성형 AI'}, 'id': 'call_J890dCbL8vZdPIzZpyn697JL', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'pdf_search', 'args': '{"query":"삼성전자 생성형 AI"}', 'id': 'call_J890dCbL8vZdPIzZpyn697JL', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_J890dCbL8vZdPIzZpyn697JL')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_call

In [17]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool[0].tool}")
    print(f'===== 도구 호출 =====')

In [18]:
def obervation_callback(observation) -> None:
    print(f'\n===== 관찰 내용 =====')
    print(f"Observation: {observation[0].observation}")
    print(f'===== 관찰 내용 =====')

In [19]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(f"{result}")
    print(f'===== 최종 답변 =====')

In [21]:
response = agent_with_chat_history.stream(
    {'input': '삼성전자가 자체 개발한 생성형 AI에 대한 정보를 문서에서 검색해서 알려주세요.'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [22]:
for step in response:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : pdf_search
===== 도구 호출 =====

Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`


SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고

In [23]:
response = agent_with_chat_history.stream(
    {'input': '이전 답변을 영어로 변역해주세요.'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [24]:
for step in response:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)



> Entering new None chain...
Here is the translation of the previous response into English:

Samsung Electronics' self-developed generative AI model, 'Samsung Gauss', has the following information:

1. **Model Composition**: Samsung Gauss consists of three models: language, code, and image, and is designed to operate on-device. This has the advantage of minimizing the risk of user information being leaked externally.

2. **Official Announcement**: Samsung Electronics first unveiled Samsung Gauss at the 'Samsung AI Forum 2023' on November 8, 2023. The model is named after the mathematician Gauss, who established the theory of normal distribution.

3. **Functions**:
   - **Language Model**: Supports tasks such as email writing, document summarization, and translation.
   - **Code Model**: Provides a conversational interface through the AI coding assistant 'Code.i', optimized for in-house software development.
   - **Image Model**: Capable of generating creative images and modifying exi